In [1]:
import pandas as pd
import os
import datetime
import numpy as np
import gc

os.getcwd()

# To check about the 2019 weekly file path df week filter for the rolling later

'/home/jian/Projects/Big_Lots/Analysis/2019_Q4/Migration_JL_status_change'

In [2]:
prior_quarter_End=datetime.date(2020,2,1)
recent_complete_quarter_End=prior_quarter_End+datetime.timedelta(days=7*13)

print("prior_quarter_End",prior_quarter_End)
print("recent_complete_quarter_End",recent_complete_quarter_End)

lapsed_date_begin=prior_quarter_End-datetime.timedelta(days=(4*365+1+6))
print("lapsed_date_begin",lapsed_date_begin)


prior_quarter_End 2020-02-01
recent_complete_quarter_End 2020-05-02
lapsed_date_begin 2016-01-26


In [3]:
lapsed=pd.read_table("/home/jian/Projects/Big_Lots/Loyal_members/loyalty_sales_data/lapsed20140826_20170226/MediaStormLapsedCustDtl.txt",
                     sep=",",usecols=['customer_id_hashed','transaction_date'],dtype=str).drop_duplicates()
lapsed=lapsed[lapsed['transaction_date']>=str(lapsed_date_begin)]

print(lapsed['transaction_date'].min())
print(lapsed['transaction_date'].max())

2015-07-28
2017-02-26


In [4]:
# change to the weekly based on the celery weekly run

chunksize_num = 10**7
filename='/home/jian/Projects/Big_Lots/Live_Ramp/Quarterly_Update_2019Q1/crm_newscore_0922/combinedtransactions_0922.csv'
dftrans_before_20180922=pd.DataFrame()
count_i=0

for chunk in pd.read_csv(filename, chunksize=chunksize_num,dtype=str,usecols=['customer_id_hashed','transaction_date']):
    chunk = chunk.drop_duplicates()
    chunk=chunk[chunk['transaction_date']>=str(lapsed_date_begin)]
    dftrans_before_20180922=dftrans_before_20180922.append(chunk)
    count_i+=1
    print(count_i,datetime.datetime.now())
dftrans_before_20180922=dftrans_before_20180922.drop_duplicates()    

1 2019-11-08 19:00:42.756875
2 2019-11-08 19:01:42.772373
3 2019-11-08 19:02:26.446788
4 2019-11-08 19:03:28.752790
5 2019-11-08 19:06:30.277716
6 2019-11-08 19:08:47.597534
7 2019-11-08 19:09:42.609846
8 2019-11-08 19:10:22.332264
9 2019-11-08 19:11:00.635467
10 2019-11-08 19:12:10.320726
11 2019-11-08 19:13:49.112202
12 2019-11-08 19:14:54.667240
13 2019-11-08 19:15:50.018085
14 2019-11-08 19:16:46.445256
15 2019-11-08 19:17:43.312855


In [5]:
all_rewards_most_recent=dftrans_before_20180922.append(lapsed)
all_rewards_most_recent=all_rewards_most_recent.sort_values(["customer_id_hashed","transaction_date"],ascending=[True,False])
all_rewards_most_recent=all_rewards_most_recent.drop_duplicates(['customer_id_hashed'])
print(all_rewards_most_recent.shape)

del chunk
del dftrans_before_20180922
gc.collect()

(22822410, 2)


0

In [6]:
def recursive_file_gen(root_path):
    for root, dirs, files in os.walk(root_path):
        for file in files:
            yield os.path.join(root,file)

In [7]:
print("all_rewards_most_recent['transaction_date'].max()",all_rewards_most_recent['transaction_date'].max())

print("all_rewards_most_recent['transaction_date'].min()",all_rewards_most_recent['transaction_date'].min())


all_rewards_most_recent['transaction_date'].max() 2018-09-22
all_rewards_most_recent['transaction_date'].min() 2015-07-28


In [8]:
# Historical    
historical_daily_data_folder="/home/jian/BigLots/hist_daily_data_itemlevel_decompressed/"
historical_daily_data_list=list(recursive_file_gen(historical_daily_data_folder))
historical_daily_data_list=[x for x in historical_daily_data_list if (".txt" in x) & ("DailySales" in x)]
historical_daily_df=pd.DataFrame({"file_path":historical_daily_data_list})
historical_daily_df['week_end_dt']=historical_daily_df['file_path'].apply(lambda x: x.split(".")[0].split("MediaStormDailySalesHistory")[1])
historical_daily_df['week_end_dt']=historical_daily_df['week_end_dt'].apply(lambda x: datetime.datetime.strptime(x,"%Y%m%d").date())
historical_daily_df=historical_daily_df[historical_daily_df['week_end_dt']<=datetime.date(2019,5,4)] # hard-code due to the 1st week of daily in item available
historical_daily_df=historical_daily_df[historical_daily_df['week_end_dt']>datetime.date(2018,9,22)] # hard-code
historical_daily_df=historical_daily_df.sort_values("week_end_dt")
print(historical_daily_df.shape)
historical_daily_df.tail(3)

print("historical_daily_df['week_end_dt'].max(): ", historical_daily_df['week_end_dt'].max())
print("historical_daily_df['week_end_dt'].min(): ", historical_daily_df['week_end_dt'].min())

(20, 2)
historical_daily_df['week_end_dt'].max():  2019-02-09
historical_daily_df['week_end_dt'].min():  2018-09-29


In [9]:
prior_quarter_End

datetime.date(2019, 8, 3)

In [10]:
recent_complete_quarter_End

datetime.date(2019, 11, 2)

In [11]:
df_historical_daily_20180929_to_before_Q=historical_daily_df[historical_daily_df['week_end_dt']<=prior_quarter_End]
print(df_historical_daily_20180929_to_before_Q.shape)


(20, 2)


In [12]:
weekly_2019_daily_df=list(recursive_file_gen("/home/jian/BigLots/"))

weekly_2019_daily_df=[x for x in weekly_2019_daily_df if "2018" not in x]
weekly_2019_daily_df=[x for x in weekly_2019_daily_df if "2017" not in x]
weekly_2019_daily_df=[x for x in weekly_2019_daily_df if "2016" not in x]
weekly_2019_daily_df=[x for x in weekly_2019_daily_df if "2016" not in x]
weekly_2019_daily_df=[x for x in weekly_2019_daily_df if "Historical" not in x]
weekly_2019_daily_df=[x for x in weekly_2019_daily_df if "hist_daily_data_itemlevel_decompressed" not in x]
weekly_2019_daily_df=[x for x in weekly_2019_daily_df if ".txt" in x]
weekly_2019_daily_df=[x for x in weekly_2019_daily_df if "aily" in x]
weekly_2019_daily_df=pd.DataFrame({"file_path":weekly_2019_daily_df})
weekly_2019_daily_df['week_end_dt']=weekly_2019_daily_df['file_path'].apply(lambda x: x.split("/MediaStorm_")[1][:10])

weekly_2019_daily_df['week_end_dt']=weekly_2019_daily_df['week_end_dt'].apply(lambda x: datetime.datetime.strptime(x,"%Y-%m-%d").date())
print(weekly_2019_daily_df.shape)
weekly_2019_daily_df.head(2)

weekly_2019_daily_df_in_recent_Q=weekly_2019_daily_df[(weekly_2019_daily_df['week_end_dt']>prior_quarter_End) & (weekly_2019_daily_df['week_end_dt']<=recent_complete_quarter_End)]
# Change to > prior_quarter_End for the next quarter
print(weekly_2019_daily_df_in_recent_Q.shape)


weekly_2019_daily_df_before_Q=weekly_2019_daily_df[weekly_2019_daily_df['week_end_dt']<=prior_quarter_End]
weekly_2019_daily_df_before_Q=weekly_2019_daily_df_before_Q[weekly_2019_daily_df_before_Q['week_end_dt']>df_historical_daily_20180929_to_before_Q['week_end_dt'].max()]
print(weekly_2019_daily_df_before_Q.shape) # Q1 should be 0, and Q2&after not

df_all_daily_files_20190922_before_quarter=df_historical_daily_20180929_to_before_Q.append(weekly_2019_daily_df_before_Q)


(44, 2)
(13, 2)
(25, 2)


In [13]:
weekly_2019_daily_df_in_recent_Q

,file_path,week_end_dt
31,/home/jian/BigLots/2019_by_weeks/MediaStorm_20...,2019-08-10
32,/home/jian/BigLots/2019_by_weeks/MediaStorm_20...,2019-08-17
33,/home/jian/BigLots/2019_by_weeks/MediaStorm_20...,2019-08-24
34,/home/jian/BigLots/2019_by_weeks/MediaStorm_20...,2019-08-31
35,/home/jian/BigLots/2019_by_weeks/MediaStorm_20...,2019-09-07
36,/home/jian/BigLots/2019_by_weeks/MediaStorm_20...,2019-09-14
37,/home/jian/BigLots/2019_by_weeks/MediaStorm_20...,2019-09-21
38,/home/jian/BigLots/2019_by_weeks/MediaStorm_20...,2019-09-28
39,/home/jian/BigLots/2019_by_weeks/MediaStorm_20...,2019-10-05
40,/home/jian/BigLots/2019_by_weeks/MediaStorm_20...,2019-10-12


In [14]:
print(df_all_daily_files_20190922_before_quarter.shape)
df_all_daily_files_20190922_before_quarter.sort_values("week_end_dt")

(45, 2)


,file_path,week_end_dt
7,/home/jian/BigLots/hist_daily_data_itemlevel_d...,2018-09-29
8,/home/jian/BigLots/hist_daily_data_itemlevel_d...,2018-10-06
9,/home/jian/BigLots/hist_daily_data_itemlevel_d...,2018-10-13
10,/home/jian/BigLots/hist_daily_data_itemlevel_d...,2018-10-20
11,/home/jian/BigLots/hist_daily_data_itemlevel_d...,2018-10-27
12,/home/jian/BigLots/hist_daily_data_itemlevel_d...,2018-11-03
13,/home/jian/BigLots/hist_daily_data_itemlevel_d...,2018-11-10
14,/home/jian/BigLots/hist_daily_data_itemlevel_d...,2018-11-17
15,/home/jian/BigLots/hist_daily_data_itemlevel_d...,2018-11-24
26,/home/jian/BigLots/hist_daily_data_itemlevel_d...,2018-12-01


In [16]:
rew_df_after_20180929_before_quarter=pd.DataFrame()
rew_df_after_recent_quarter_only=pd.DataFrame()


for file in df_all_daily_files_20190922_before_quarter['file_path'].tolist():
    df=pd.read_table(file,sep="|",dtype=str,usecols=["customer_id_hashed","transaction_dt"]).drop_duplicates().rename(columns={"transaction_dt":"transaction_date"})
    df=df[~pd.isnull(df['customer_id_hashed'])]
    rew_df_after_20180929_before_quarter=rew_df_after_20180929_before_quarter.append(df)
    print(file,datetime.datetime.now())

for file in weekly_2019_daily_df_in_recent_Q['file_path'].tolist():
    df=pd.read_table(file,sep="|",dtype=str,usecols=["customer_id_hashed","transaction_dt"]).drop_duplicates().rename(columns={"transaction_dt":"transaction_date"})
    df=df[~pd.isnull(df['customer_id_hashed'])]
    rew_df_after_recent_quarter_only=rew_df_after_recent_quarter_only.append(df)
    print(file,datetime.datetime.now())


/home/jian/BigLots/hist_daily_data_itemlevel_decompressed/MediaStormDailySalesHistory20180929.txt 2019-11-08 20:09:26.459494
/home/jian/BigLots/hist_daily_data_itemlevel_decompressed/MediaStormDailySalesHistory20181006.txt 2019-11-08 20:09:55.872787
/home/jian/BigLots/hist_daily_data_itemlevel_decompressed/MediaStormDailySalesHistory20181013.txt 2019-11-08 20:10:15.932471
/home/jian/BigLots/hist_daily_data_itemlevel_decompressed/MediaStormDailySalesHistory20181020.txt 2019-11-08 20:10:36.410932
/home/jian/BigLots/hist_daily_data_itemlevel_decompressed/MediaStormDailySalesHistory20181027.txt 2019-11-08 20:10:59.952289
/home/jian/BigLots/hist_daily_data_itemlevel_decompressed/MediaStormDailySalesHistory20181103.txt 2019-11-08 20:11:24.056631
/home/jian/BigLots/hist_daily_data_itemlevel_decompressed/MediaStormDailySalesHistory20181110.txt 2019-11-08 20:11:49.314335
/home/jian/BigLots/hist_daily_data_itemlevel_decompressed/MediaStormDailySalesHistory20181117.txt 2019-11-08 20:12:20.132995


In [17]:
all_rewards_most_recent=all_rewards_most_recent.append(rew_df_after_20180929_before_quarter)
all_rewards_most_recent=all_rewards_most_recent.sort_values(["customer_id_hashed","transaction_date"],ascending=[True,False])
all_rewards_most_recent=all_rewards_most_recent.drop_duplicates(['customer_id_hashed'])
all_rewards_most_recent['transaction_date_before_the_Quarter']=all_rewards_most_recent['transaction_date']

all_rewards_most_recent_After=all_rewards_most_recent[['customer_id_hashed','transaction_date']].append(rew_df_after_recent_quarter_only)
all_rewards_most_recent_After=all_rewards_most_recent_After.sort_values(["customer_id_hashed","transaction_date"],ascending=[True,False])
all_rewards_most_recent_After=all_rewards_most_recent_After.drop_duplicates(['customer_id_hashed']).rename(columns={"transaction_date":"transaction_date_after_the_Quarter"})

all_rewards_most_recent=pd.merge(all_rewards_most_recent,all_rewards_most_recent_After,on="customer_id_hashed",how="outer")
del all_rewards_most_recent_After

gc.collect()

/home/jian/.local/lib/python3.6/site-packages/pandas/core/frame.py:7138: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  sort=sort,


0

In [18]:
all_rewards_most_recent['transaction_date_before_the_Quarter']=all_rewards_most_recent['transaction_date_before_the_Quarter'].fillna("2099-12-31")
all_rewards_most_recent['transaction_date_before_the_Quarter']=all_rewards_most_recent['transaction_date_before_the_Quarter'].astype(str)

# Change here to use unique dates and merge
all_rewards_most_recent_date=all_rewards_most_recent[['transaction_date_before_the_Quarter']].drop_duplicates()
all_rewards_most_recent_date['date']=pd.to_datetime(all_rewards_most_recent_date['transaction_date_before_the_Quarter'],format="%Y-%m-%d").dt.date
all_rewards_most_recent_date['Days_to_pre_Quarter_End']=prior_quarter_End-all_rewards_most_recent_date['date']
all_rewards_most_recent_date['Month_to_pre_Quarter_End']=all_rewards_most_recent_date['Days_to_pre_Quarter_End'].apply(lambda x: x.days/(365.25/12))
all_rewards_most_recent_date['Group_before_the_Quarter']=np.where((all_rewards_most_recent_date['Month_to_pre_Quarter_End']>12) & (all_rewards_most_recent_date['Month_to_pre_Quarter_End']<=48),"Lapsed_13_48",
                                                       np.where(all_rewards_most_recent_date['Month_to_pre_Quarter_End']>48,"WD_48+",
                                                                np.where((all_rewards_most_recent_date['Month_to_pre_Quarter_End']>=0) & (all_rewards_most_recent_date['Month_to_pre_Quarter_End']<=12),"Active",
                                                                        np.where(all_rewards_most_recent_date['Month_to_pre_Quarter_End']<0,"NotAvailable_Before_Quarter","NaN")
                                                                        )
                                                               )
                                                       )
all_rewards_most_recent_date=all_rewards_most_recent_date[['transaction_date_before_the_Quarter','Group_before_the_Quarter']]
all_rewards_most_recent=pd.merge(all_rewards_most_recent,all_rewards_most_recent_date,on="transaction_date_before_the_Quarter",how="left")

In [19]:
recent_complete_quarter_End

datetime.date(2019, 11, 2)

In [20]:
all_rewards_most_recent['transaction_date_after_the_Quarter']=all_rewards_most_recent['transaction_date_after_the_Quarter'].fillna("2099-12-31")
all_rewards_most_recent['transaction_date_after_the_Quarter']=all_rewards_most_recent['transaction_date_after_the_Quarter'].astype(str)

all_rewards_most_recent_date=all_rewards_most_recent[['transaction_date_after_the_Quarter']].drop_duplicates()
all_rewards_most_recent_date['date']=pd.to_datetime(all_rewards_most_recent_date['transaction_date_after_the_Quarter'],format="%Y-%m-%d").dt.date
all_rewards_most_recent_date['Days_to_recentt_Quarter_End']=recent_complete_quarter_End-all_rewards_most_recent_date['date']
all_rewards_most_recent_date['Month_to_recent_Quarter_End']=all_rewards_most_recent_date['Days_to_recentt_Quarter_End'].apply(lambda x: x.days/(365.25/12))
all_rewards_most_recent_date['Group_after_the_Quarter']=np.where((all_rewards_most_recent_date['Month_to_recent_Quarter_End']>12) & (all_rewards_most_recent_date['Month_to_recent_Quarter_End']<=48),"Lapsed_13_48",
                                                       np.where(all_rewards_most_recent_date['Month_to_recent_Quarter_End']>48,"WD_48+",
                                                                np.where((all_rewards_most_recent_date['Month_to_recent_Quarter_End']>=0) & (all_rewards_most_recent_date['Month_to_recent_Quarter_End']<=12),"Active",
                                                                        np.where(all_rewards_most_recent_date['Month_to_recent_Quarter_End']<0,"NotAvailable_Before_Q4","NaN")
                                                                        )
                                                               )
                                                       )

all_rewards_most_recent_date=all_rewards_most_recent_date[['transaction_date_after_the_Quarter','Group_after_the_Quarter']]
all_rewards_most_recent=pd.merge(all_rewards_most_recent,all_rewards_most_recent_date,on="transaction_date_after_the_Quarter",how="left")

In [21]:
all_rewards_most_recent['Group_before_the_Quarter'].unique().tolist()

['Active', 'Lapsed_13_48', 'WD_48+', 'NotAvailable_Before_Quarter']

In [22]:
all_rewards_most_recent['Group_after_the_Quarter'].unique().tolist()

['Lapsed_13_48', 'Active', 'WD_48+']

In [23]:
all_rewards_most_recent[all_rewards_most_recent['Group_before_the_Quarter']=="Active"].shape

(18294410, 6)

In [24]:
print(datetime.datetime.now())
# stop here

2019-11-08 21:22:53.588598


In [25]:
RecentQuarter_sales_by_id=pd.DataFrame()
print(weekly_2019_daily_df_in_recent_Q.shape)
for file in weekly_2019_daily_df_in_recent_Q['file_path'].tolist():
    df=pd.read_table(file,sep="|",dtype=str,
                     usecols=['location_id','transaction_dt','transaction_id','customer_id_hashed','item_transaction_amt'])
    df=df[~pd.isnull(df['customer_id_hashed'])]
    df['item_transaction_amt']=df['item_transaction_amt'].astype(float)
    df_sales=df.groupby(['customer_id_hashed'])['item_transaction_amt'].sum().to_frame().reset_index().rename(columns={"item_transaction_amt":"sales_recent_Quarter"})
    
    df_trans=df[['location_id','transaction_dt','transaction_id','customer_id_hashed']].drop_duplicates()
    df_trans=df_trans.groupby(["customer_id_hashed"])['transaction_id'].count().to_frame().reset_index().rename(columns={"transaction_id":"trans_recent_Quarter"})
    
    df=pd.merge(df_sales,df_trans,on="customer_id_hashed",how="outer")
    
    RecentQuarter_sales_by_id=RecentQuarter_sales_by_id.append(df)
    print(file,datetime.datetime.now())
RecentQuarter_sales_by_id=RecentQuarter_sales_by_id.groupby("customer_id_hashed")['sales_recent_Quarter','trans_recent_Quarter'].sum().reset_index()
RecentQuarter_sales_by_id['Recent_Quarter_Shopping_Group']="Recent_Quarter_Shopped"

(13, 2)
/home/jian/BigLots/2019_by_weeks/MediaStorm_2019-08-10/MediaStormDailySales20190813-120541-143.txt 2019-11-08 21:23:59.637573
/home/jian/BigLots/2019_by_weeks/MediaStorm_2019-08-17/MediaStormDailySales20190820-113628-440.txt 2019-11-08 21:25:02.380208
/home/jian/BigLots/2019_by_weeks/MediaStorm_2019-08-24/MediaStormDailySales20190827-113630-114.txt 2019-11-08 21:26:04.085535
/home/jian/BigLots/2019_by_weeks/MediaStorm_2019-08-31/MediaStormDailySales20190903-113649-462.txt 2019-11-08 21:27:12.703217
/home/jian/BigLots/2019_by_weeks/MediaStorm_2019-09-07/MediaStormDailySales20190910-120643-163.txt 2019-11-08 21:27:59.596681
/home/jian/BigLots/2019_by_weeks/MediaStorm_2019-09-14/MediaStormDailySales20190917-110425-322.txt 2019-11-08 21:29:08.456289
/home/jian/BigLots/2019_by_weeks/MediaStorm_2019-09-21/MediaStormDailySales20190924-112126-506.txt 2019-11-08 21:30:16.492442
/home/jian/BigLots/2019_by_weeks/MediaStorm_2019-09-28/MediaStormDailySales20191001-111241-312.txt 2019-11-08 

In [26]:
all_rewards_most_recent=pd.merge(all_rewards_most_recent,RecentQuarter_sales_by_id,on="customer_id_hashed",how="outer")
all_rewards_most_recent['Group_before_the_Quarter']=all_rewards_most_recent['Group_before_the_Quarter'].fillna("NotAvailable_Before_Quarter")
all_rewards_most_recent['Group_after_the_Quarter']=all_rewards_most_recent['Group_after_the_Quarter'].fillna("Active")

all_rewards_most_recent.shape

(29749790, 9)

In [27]:
Recent_Quarter_new_sign_ups_files=list(recursive_file_gen("/home/jian/BigLots/"))

Recent_Quarter_new_sign_ups_files=[x for x in Recent_Quarter_new_sign_ups_files if "Master" in x]
Recent_Quarter_new_sign_ups_files=[x for x in Recent_Quarter_new_sign_ups_files if "2018" not in x]
Recent_Quarter_new_sign_ups_files=[x for x in Recent_Quarter_new_sign_ups_files if "2017" not in x]
Recent_Quarter_new_sign_ups_files=[x for x in Recent_Quarter_new_sign_ups_files if "2016" not in x]
Recent_Quarter_new_sign_ups_files=[x for x in Recent_Quarter_new_sign_ups_files if ".txt" in x]

Recent_Quarter_new_sign_ups_files_df=pd.DataFrame({"file_path":Recent_Quarter_new_sign_ups_files})
Recent_Quarter_new_sign_ups_files_df['week_end_dt']=Recent_Quarter_new_sign_ups_files_df['file_path'].apply(lambda x: x.split("/MediaStorm_")[1][:10])
Recent_Quarter_new_sign_ups_files_df=Recent_Quarter_new_sign_ups_files_df[Recent_Quarter_new_sign_ups_files_df['week_end_dt']<=str(recent_complete_quarter_End)]
Recent_Quarter_new_sign_ups_files_df=Recent_Quarter_new_sign_ups_files_df[Recent_Quarter_new_sign_ups_files_df['week_end_dt']>str(prior_quarter_End)]
print(Recent_Quarter_new_sign_ups_files_df.shape)
Recent_Quarter_new_sign_ups_files_df.head(2)

(13, 2)


,file_path,week_end_dt
29,/home/jian/BigLots/2019_by_weeks/MediaStorm_20...,2019-08-10
30,/home/jian/BigLots/2019_by_weeks/MediaStorm_20...,2019-08-17


In [28]:
Recent_Quarter_new_sign_ups_files_df

,file_path,week_end_dt
29,/home/jian/BigLots/2019_by_weeks/MediaStorm_20...,2019-08-10
30,/home/jian/BigLots/2019_by_weeks/MediaStorm_20...,2019-08-17
31,/home/jian/BigLots/2019_by_weeks/MediaStorm_20...,2019-08-24
32,/home/jian/BigLots/2019_by_weeks/MediaStorm_20...,2019-08-31
33,/home/jian/BigLots/2019_by_weeks/MediaStorm_20...,2019-09-07
34,/home/jian/BigLots/2019_by_weeks/MediaStorm_20...,2019-09-14
35,/home/jian/BigLots/2019_by_weeks/MediaStorm_20...,2019-09-21
36,/home/jian/BigLots/2019_by_weeks/MediaStorm_20...,2019-09-28
37,/home/jian/BigLots/2019_by_weeks/MediaStorm_20...,2019-10-05
38,/home/jian/BigLots/2019_by_weeks/MediaStorm_20...,2019-10-12


In [29]:
all_new_sign_ups=pd.DataFrame()
for file in Recent_Quarter_new_sign_ups_files_df['file_path'].unique().tolist():
    df=pd.read_table(file,sep="|",usecols=['customer_id_hashed','sign_up_date'])
    all_new_sign_ups=all_new_sign_ups.append(df)
all_new_sign_ups=all_new_sign_ups.sort_values(['customer_id_hashed','sign_up_date'])
all_new_sign_ups=all_new_sign_ups.drop_duplicates("customer_id_hashed")
print(all_new_sign_ups.shape)

all_new_sign_ups['NewRewards']="Recent_Quarter_New_Sign_Ups"
all_new_sign_ups=all_new_sign_ups.drop_duplicates("customer_id_hashed")

(1587076, 2)


In [30]:
all_rewards_most_recent=pd.merge(all_rewards_most_recent,all_new_sign_ups,on="customer_id_hashed",how="outer")
all_rewards_most_recent['NewRewards']=all_rewards_most_recent['NewRewards'].fillna("Old_Rewards_Members")

all_rewards_most_recent['sales_recent_Quarter']=all_rewards_most_recent['sales_recent_Quarter'].fillna(0)
all_rewards_most_recent['trans_recent_Quarter']=all_rewards_most_recent['trans_recent_Quarter'].fillna(0)

all_rewards_most_recent['Recent_Quarter_Shopping_Group']=all_rewards_most_recent['Recent_Quarter_Shopping_Group'].fillna("Recent_Quarter_No_Shop")

all_rewards_most_recent.shape

(30241196, 11)

In [36]:
all_rewards_most_recent['Group_before_the_Quarter']=all_rewards_most_recent['Group_before_the_Quarter'].fillna("nan")
all_rewards_most_recent['Group_after_the_Quarter']=all_rewards_most_recent['Group_after_the_Quarter'].fillna("nan")
all_rewards_most_recent['NewRewards']=all_rewards_most_recent['NewRewards'].fillna("nan")
all_rewards_most_recent['Recent_Quarter_Shopping_Group']=all_rewards_most_recent['Recent_Quarter_Shopping_Group'].fillna("nan")

In [37]:
all_rewards_most_recent['sales_recent_Quarter']=all_rewards_most_recent['sales_recent_Quarter'].fillna(0)
all_rewards_most_recent['trans_recent_Quarter']=all_rewards_most_recent['trans_recent_Quarter'].fillna(0)


In [38]:
output_id_count=all_rewards_most_recent.groupby(['Group_before_the_Quarter','Group_after_the_Quarter','NewRewards','Recent_Quarter_Shopping_Group'])['customer_id_hashed'].count().to_frame().reset_index().rename(columns={"customer_id_hashed":"id_count"})
output_id_sales=all_rewards_most_recent.groupby(['Group_before_the_Quarter','Group_after_the_Quarter','NewRewards','Recent_Quarter_Shopping_Group'])['sales_recent_Quarter','trans_recent_Quarter'].sum().reset_index()

output=pd.merge(output_id_count,output_id_sales,on=['Group_before_the_Quarter','Group_after_the_Quarter','NewRewards','Recent_Quarter_Shopping_Group'],how="outer")
output

,Group_before_the_Quarter,Group_after_the_Quarter,NewRewards,Recent_Quarter_Shopping_Group,id_count,sales_recent_Quarter,trans_recent_Quarter
0,Active,Active,Old_Rewards_Members,Recent_Quarter_No_Shop,9994085,0.000000e+00,0.0
1,Active,Active,Old_Rewards_Members,Recent_Quarter_Shopped,6739182,5.712362e+08,17005153.0
2,Active,Active,Recent_Quarter_New_Sign_Ups,Recent_Quarter_No_Shop,20,0.000000e+00,0.0
3,Active,Active,Recent_Quarter_New_Sign_Ups,Recent_Quarter_Shopped,42,8.147450e+03,166.0
4,Active,Lapsed_13_48,Old_Rewards_Members,Recent_Quarter_No_Shop,1561081,0.000000e+00,0.0
5,Lapsed_13_48,Active,Old_Rewards_Members,Recent_Quarter_Shopped,683412,4.918085e+07,1023580.0
6,Lapsed_13_48,Lapsed_13_48,Old_Rewards_Members,Recent_Quarter_No_Shop,9213639,0.000000e+00,0.0
7,Lapsed_13_48,Lapsed_13_48,Recent_Quarter_New_Sign_Ups,Recent_Quarter_No_Shop,1,0.000000e+00,0.0
8,Lapsed_13_48,WD_48+,Old_Rewards_Members,Recent_Quarter_No_Shop,118825,0.000000e+00,0.0
9,NotAvailable_Before_Quarter,Active,Old_Rewards_Members,Recent_Quarter_Shopped,336591,2.920117e+07,516294.0


In [39]:
os.getcwd()

'/home/jian/Projects/Big_Lots/Analysis/2019_Q4/Migration_JL_status_change'

In [40]:
# Stop here

In [41]:
output['Recent_Quarteer_Sale_per_ID']=output['sales_recent_Quarter']/output['id_count']
output['Recent_Quarteer_Sale_per_ID']=output['Recent_Quarteer_Sale_per_ID'].apply(lambda x: np.round(x,2))

output['inclusion/exclusion']=np.where((output['Group_before_the_Quarter']!="NotAvailable_Before_Quarter") & (output['NewRewards']=="Recent_Quarter_New_Sign_Ups"),"Exclusion","Inclusion")


In [42]:
output.to_csv("./BL_2019Q3_rewards_migration_output_summary_JL_"+str(datetime.datetime.now().date())+".csv",index=False)



In [43]:
all_rewards_most_recent.to_csv("./BL_2019Q3_rewards_migration_rewards_data_JL_"+str(datetime.datetime.now().date())+".csv",index=False)


In [50]:
output['id_count'].sum()

30241196

In [44]:
all_rewards_most_recent.shape

(30241196, 11)

In [45]:
all_rewards_most_recent['customer_id_hashed'].nunique()

30241196

In [46]:
all_rewards_most_recent.shape

(30241196, 11)

In [47]:
all_rewards_most_recent.head(100)

,customer_id_hashed,transaction_date,transaction_date_before_the_Quarter,transaction_date_after_the_Quarter,Group_before_the_Quarter,Group_after_the_Quarter,sales_recent_Quarter,trans_recent_Quarter,Recent_Quarter_Shopping_Group,sign_up_date,NewRewards
0,00000135f48c68690ad3d5fc9ada41bb5cd687452007e8...,2018-10-30,2018-10-30,2018-10-30,Active,Lapsed_13_48,0.00,0.0,Recent_Quarter_No_Shop,NaN,Old_Rewards_Members
1,000001dadc0265bf9d250566d74e0006323f18b5826641...,2018-12-21,2018-12-21,2019-10-12,Active,Active,53.95,2.0,Recent_Quarter_Shopped,NaN,Old_Rewards_Members
2,000003066bafc6b19c55d3be5fc646a253c659e9e6baea...,2018-11-12,2018-11-12,2019-10-15,Active,Active,17.00,1.0,Recent_Quarter_Shopped,NaN,Old_Rewards_Members
3,0000034fec3c2cbbf7a2b38fbf80d84abc72d8bed269c4...,2019-05-25,2019-05-25,2019-05-25,Active,Active,0.00,0.0,Recent_Quarter_No_Shop,NaN,Old_Rewards_Members
4,00000435bfb3bf42e3beb4c9b3942c552d09f0e49e5a75...,2019-06-30,2019-06-30,2019-10-12,Active,Active,14.19,2.0,Recent_Quarter_Shopped,NaN,Old_Rewards_Members
...,...,...,...,...,...,...,...,...,...,...,...
95,0000358fbc9f7162c1acbfc88f1211ebec1245400631f1...,2019-07-23,2019-07-23,2019-10-12,Active,Active,116.79,6.0,Recent_Quarter_Shopped,NaN,Old_Rewards_Members
96,00003654ed0884109390963c2ba2c4b61b49442904a019...,2019-07-02,2019-07-02,2019-10-20,Active,Active,218.67,5.0,Recent_Quarter_Shopped,NaN,Old_Rewards_Members
97,000039d8d4585c0eb6e5ff78217d3769807fb5a9b08158...,2016-03-14,2016-03-14,2016-03-14,Lapsed_13_48,Lapsed_13_48,0.00,0.0,Recent_Quarter_No_Shop,NaN,Old_Rewards_Members
98,00003a96d2cf6a9edc5f60880aa4445d5e04deb8f26656...,2019-03-10,2019-03-10,2019-10-09,Active,Active,78.50,4.0,Recent_Quarter_Shopped,NaN,Old_Rewards_Members


In [48]:
all_rewards_most_recent[all_rewards_most_recent['Group_before_the_Quarter']=="Active"].shape

(18294410, 11)